In [12]:
from data_loader import *
from utils import *

from concurrent.futures import ThreadPoolExecutor,as_completed
from itertools import product

In [13]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']].head()

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil


In [14]:
teams_data = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")
teams_data = teams_data.merge(leagues_dat,left_on='league',right_on="league_id",how = "left")
teams_data.head()


,team_name,league,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
0,Angers,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
1,Lille,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
2,Lyon,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
3,Marseille,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,Nantes,61,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg


In [ ]:
read_all_teams = False
unique_teams = teams_data[teams_data.league_id == 39]['team_name'].unique()
unique_teams = ['Liverpool']

if read_all_teams:
    # All teams:
    unique_teams = teams_data['team_name'].unique()


# Specific teams:
print("Expected API requests: ",int(len(unique_teams) * 38 * .8))

In [ ]:
teams = unique_teams
season = [2023]
team_season_pair = list(product(teams,season))

In [ ]:
team_season_pair

In [15]:
team = "Chelsea"
season = 2025

In [16]:
dat = read_fixtures_for_season(team,season,sleep_time=60)

processing for Chelsea, 2025
Pulling for Chelsea,2025 with league_id: 39


/home/opc/OverperformXG/app/data_loader.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_fixtures['winner'] = np.where(team_fixtures.teams_home_winner == True,team_fixtures.teams_home_name,
/home/opc/OverperformXG/app/data_loader.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/home/opc/OverperformXG/app/data_

In [17]:
liverpool_dat = pd.read_parquet("/home/opc/OverperformXG/data/Fixtures/Chelsea_2025.parquet")

In [19]:
liverpool_dat.fixture_id.unique()

array([1378976, 1378991, 1379021, 1379033, 1378988, 1379001, 1379015])

In [ ]:
parallel = False
workers = 3
visited_list = list()

if parallel:
    print(f"Running in parallel with {workers} workers")
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {executor.submit(read_fixtures_for_season,t,s) : (t,s) for t,s in team_season_pair}
        
else:
    for team in teams:
        try:
            dat = read_fixtures_for_season(team,season,sleep_time=60)
            visited_list.append(team)
        except:
            print(f"{team} does not have matches in {season}")

In [ ]:
teams = list(set(teams) - set(visited_list))
season = [2025]
team_season_pair = list(product(teams,season))

team_season_pair



In [ ]:
len(teams)

In [ ]:
team_season_pair[0:3]